## Revolut FinCrime Challenge: Data Analysis

There will be a notebook for each one of the Machine Learning Pipeline steps:

1. Data Analysis
2. Feature Engineering
3. Feature Selection
4. Model Building

**This is the notebook for step 1: Data Analysis**


## Predicting Fradulent transactions

The aim of the project is to build a machine learning model to find the fraudsters and take appropriate actions.

![SegmentLocal](fraud.gif "segment")

### Why is this important? 

Fraudsters can use our App to steal the other people's money from outside into an account via Top-Up's. So finding them and blocking them is very necessary.


**We will analyse the datasets to identify:**

1. [Missing values](#missing)<br>
2. [Numerical variables](#numvar)<br>
3. [Temporal variables](#temvar)<br>
4. [Categorical variables](#catvar)<br>
7. [Cardinality of the categorical variables](#cardi)<br>
8. [Rare Labels](#rarelab)<br>

In [2]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 90
% matplotlib inline

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

# to ignore the warnings
import warnings
warnings.filterwarnings('ignore')

import itertools
from itertools import cycle,islice
%precision %.2f

UsageError: Line magic function `%` not found.


In [ ]:
# Load the datasets
df_transactions = pd.read_csv('../data/transactions.csv')
df_users = pd.read_csv('../data/users.csv')
df_fraudsters = pd.read_csv('../data/fraudsters.csv')

# Rows and Columns in the Datasets
print(df_transactions.shape)
print(df_users.shape)
print(df_fraudsters.shape)

In [ ]:
# Checking the data
df_transactions.head()

In [ ]:
# Checking the data
df_users.head()

In [ ]:
# Checking the data
df_fraudsters.head()

In [ ]:
# Converting the names of the columns to lower case
df_transactions.columns = map(str.lower, df_transactions.columns)
df_users.columns = map(str.lower, df_users.columns)
df_fraudsters.columns = map(str.lower, df_fraudsters.columns)

In [ ]:
# Checking the column names
df_transactions.columns

In [ ]:
df_user_transactions = df_transactions.merge(df_users,how='left',left_on='user_id',right_on='id')

In [ ]:
# Renaming the columns
df_user_transactions.rename({'id_x': 'transaction_id', 'created_date_x': 'transaction_date','created_date_y':'registered_date'}, axis=1, inplace=True)
df_user_transactions.drop(['id_y'],axis = 1,inplace=True)

In [ ]:
df_user_transactions.head()

In [ ]:
df_user_transactions.info()

In [ ]:
# Changing the datatypes of temporal columns to datatime.
df_user_transactions[["transaction_date", "registered_date", "birth_date"]] = df_user_transactions[["transaction_date", "registered_date", "birth_date"]].apply(pd.to_datetime)

In [ ]:
df_user_transactions[["transaction_date", "registered_date", "birth_date"]].info()

In [ ]:
# Creating the target variable 'is_fraud' by looking for the id's in the fraudsters table.
df_user_transactions['is_fraud'] = 0
df_fraudsters['is_fraud'] = 1

df_user_transactions['is_fraud'] = df_user_transactions['user_id'].isin(df_fraudsters['user_id'])
df_user_transactions['is_fraud'][df_user_transactions['is_fraud'] == 1] = 1
df_user_transactions['is_fraud'] = df_user_transactions['is_fraud'].astype(int)

In [ ]:
df_user_transactions.head()

### Missing Values
<a id="missing"></a>

In [ ]:
# make a list of the variables that contain missing values
df_user_transactions.isnull().sum()

### Numerical Variables
<a id="numvar"></a>

Let's go ahead and find the distribution of the continuous variables. We will consider continuous all those that are not temporal or discrete variables in our dataset.

In [ ]:
cont_vars = [var for var in df_user_transactions.columns if df_user_transactions[var].dtypes!='O' and 'date' not in var and var!='is_fraud']

In [ ]:
cont_vars

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
df_user_transactions['amount_gbp'].describe()

In [ ]:
df_user_transactions.boxplot(column = ['amount_gbp']) # Box plot

There are few transactions where amounts are significantly higher.

### Temporal variables
<a id="temvar"></a>

In [ ]:
# list of variables that contain date information
date_vars = [var for var in df_user_transactions.columns if 'date' in var]

date_vars

In the next notebook(Step2_FeatureEngineering) I tried to create some new features from these raw variables.

### Categorical Variables
<a id="catvar"></a>

Let's go ahead and find which variables are categorical.

In [ ]:
### Categorical variables

cat_vars = [var for var in df_user_transactions.columns if df_user_transactions[var].dtypes=='O' and 'date' not in var]

print('Number of categorical variables: ', len(cat_vars))

In [ ]:
# let's visualise the discrete variables
df_user_transactions[cat_vars].head()

#### Number of labels: cardinality

Let's evaluate how many different categories are present in each of the variables.

In [ ]:
for var in cat_vars:
    print(var, 'has',len(df_user_transactions[var].unique()), ' categories')

As we can see that transacition_id and user_id has significantly high dimensions, we don't consider them while building the model.

In [ ]:
def analyse_categorical(df, var):
    """
    Plotting the distribution of categorical variables.
    
    """
    if var not in ['transaction_id','user_id']:
        df[df['is_fraud'] == 1].groupby(var).count()["is_fraud"].plot(kind = "bar",color='green')
        plt.show()

In [ ]:
for var in ['type','state','country']:
    analyse_categorical(df_user_transactions, var)

From the above plots we can observe the following
 -  Fraud transactions are happening more through TOP-UP( while receiving money externally into a Revolut account) followed by a TRANSFER(sending money externally into a bank account) types.
 -  Fraud transactions are happening more in Great Britain, France and Italy.
 -  Most of the feaud transactions are completed.

But from these we cannout deduce the conclusions as it might be the case that most of the transactions are happening through TOP-UP and hence the number of frauds are high in that. So lets compare the percentage of non-fradulent vs fradulent transactions for each of these categories.

In [ ]:
def percentage_distribution(df,var):
    """
    Plotting the percentage distrubution of each variable.
    """
    counter = df.groupby(var)['is_fraud'].value_counts().unstack()
    percentage_dist = 100 * counter.divide(counter.sum(axis = 1), axis = 0)
    percentage_dist = percentage_dist[percentage_dist> 0]
    ax = percentage_dist.plot.bar(stacked=True,rot = 0,figsize = (15,5))
    plt.legend(bbox_to_anchor=(0, 1), loc='upper right', ncol=1)
    sns.set(font_scale=1)
    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        ax.text(x+width/2, 
                y+height/2, 
                '{:.2f} %'.format(height), 
                horizontalalignment='center', 
                verticalalignment='center')

In [ ]:
for var in ['type','state']:
    percentage_distribution(df_user_transactions, var)

From the above graph's we can infer that percentage of feaud transactions are high in 'ATM' and 'TRANSFER' category compared to 'TOP-UP'.

Let's analyse 'country' variable in detail.

In [ ]:
pd.options.display.float_format = '{:,.2f}%'.format
counter = df_user_transactions.groupby('country')['is_fraud'].value_counts().unstack()
#calculate the % for each group 
percentage_dist = 100 * counter.divide(counter.sum(axis = 1), axis = 0)
percentage_dist[percentage_dist[1]>0].sort_values(1,ascending=False)

From the above table we can infer that countries 'GP'(Guadeloupe),'IT'(Italy),'RE'(Reunion),'FR'(France),'NL'(Holland),'GB'(Great Britain),'DE'(Germany) have the highest fraud rate compared to others.

Let's analyse 'country' variable in detail.

In [ ]:
pd.options.display.float_format = '{:,.2f}%'.format
counter = df_user_transactions.groupby('currency')['is_fraud'].value_counts().unstack()
#calculate the % for each group
percentage_dist = 100 * counter.divide(counter.sum(axis = 1), axis = 0)
percentage_dist[percentage_dist[1]>0.1].sort_values(1,ascending=False)

From the above table we can infer that transactions which are happening in currencies 'XOF','GBP','MDL','AED','MAD' have high fraud rate compared to others.

### Rare labels:
<a id="rarelab"></a>

Let's go ahead and investigate now if there are labels that are present only in a small number of transactions. In the next section I discussed how to handle them.

In [ ]:
pd.options.display.float_format = '{:,.5f}'.format
def analyse_rare_labels(df, var, rare_perc):
    df = df.copy()
    tmp = df.groupby(var)['is_fraud'].count() / len(df)
    return tmp[tmp<rare_perc]

for var in cat_vars:
    if var not in ['transaction_id','user_id']:
        print(analyse_rare_labels(df_user_transactions, var, 0.0001))
        print()

### References

https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

https://stats.stackexchange.com/questions/384833/adjusting-probability-threshold-for-sklearns-logistic-regression-model

https://github.com/trainindata/deploying-machine-learning-models

https://medium.com/datadriveninvestor/rethinking-the-right-metrics-for-fraud-detection-4edfb629c423